In [32]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
import os, glob
from datetime import datetime, timedelta

In [33]:
def read_csv_to_df(bucket, key, encoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(encoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

meta_key = 'meta_file.csv'
bucket_name_trg = 'xetra-112233'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(bucket_name_trg)

df_meta = read_csv_to_df(bucket_trg, meta_key)

In [34]:
df_meta

,source_date,datetime_of_processing
0,2021-04-23,2021-04-23 12:33:23
1,2021-04-21,2021-04-21 12:30:21


In [35]:
arg_date = '2021-04-23'
today_str = '2021-04-25'
src_format = '%Y-%m-%d'
#start: arg_date - 1 : run from previous 1 day
start = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)  
today = datetime.strptime(today_str, src_format).date()
dates = [start + timedelta(days=x) for x in range(0, (today - start).days + 1)]

In [36]:
dates

[datetime.date(2021, 4, 22),
 datetime.date(2021, 4, 23),
 datetime.date(2021, 4, 24),
 datetime.date(2021, 4, 25)]

In [37]:
src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)

In [38]:
src_dates

{datetime.date(2021, 4, 21), datetime.date(2021, 4, 23)}

In [39]:
dates_missing = set(dates[1:]) - src_dates

In [40]:
dates_missing

{datetime.date(2021, 4, 24), datetime.date(2021, 4, 25)}

In [41]:
min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
min_date

datetime.date(2021, 4, 23)

In [42]:
dates_missing = set(dates[1:]) - src_dates
if dates_missing:
    min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
    return_dates = [date.strftime('%Y-%m-%d') for date in dates if date >= min_date]
    return_min_date = (min_date + timedelta(days=1)).strftime('%Y-%m-%d')
else:
    return_dates = []
    return_min_date = datetime(2200, 1, 1).date()

In [43]:
return_dates

['2021-04-23', '2021-04-24', '2021-04-25']

In [44]:
return_min_date

'2021-04-24'

In [45]:
# Adapter Layer

def read_csv_to_df(bucket, key, encoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(encoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    
def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [46]:
# Application layer
# Core

def extract(bucket, date_list):
    
    def csv_to_df_nb(key):
#        print(key)
        df = read_csv_to_df(bucket, key)
        return df
    
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat(map(csv_to_df_nb, files), ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    df.reset_index(inplace=True, drop=True)
    return df

def load(bucket, df, trg_key, trg_format):
    target_key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, target_key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, trg_key, trg_format, columns, arg_date):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True

In [47]:
# Application Layer - not core

def return_date_list(bucket, arg_date):
    start = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(0, (today - start).days + 1)]
    return return_date_list

In [48]:
# main function entrypoint

def main():
    
    # Parameters/Configurations
    # later read config
    arg_date = '2022-01-04'
    columns = ['ISIN', 'Mnemonic', 'Date', 'Time', 'StartPrice', 'EndPrice', 'MinPrice', 'MaxPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    bucket_name_src = 'demo-s3-nhhung2'
    bucket_name_trg = 'xetra-112233'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(bucket_name_src)
    bucket_trg = s3.Bucket(bucket_name_trg)
    
    #run application
    date_lst = return_date_list(bucket, arg_date)
    b=[]
    for i in date_lst:
        k='data/'+ i;
        b.append(k)    
    date_list=b    
    # date_list = return_date_list(bucket_src, arg_date)
    etl_report1(bucket_src, bucket_trg, date_list, trg_key, trg_format, columns, arg_date)

TEST

In [49]:
# ## test
# arg_date = '2022-01-04'
# s3 = boto3.resource('s3')
# bucket = s3.Bucket('demo-s3-nhhung2')


# date_lst = return_date_list(bucket, arg_date)
# b=[]
# for i in date_lst:
#     k='data/'+ i;
#     b.append(k)

# date_list=b 

# # print(date_list)
# # ##Test
# s3 = boto3.resource('s3')
# bucket = s3.Bucket('demo-s3-nhhung2')


# def extract(bucket, date_list):
    
#     def csv_to_df_nb(key):
#         df = read_csv_to_df(bucket, key)
#         return df
    
#     files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
#     df = pd.concat(map(csv_to_df_nb, files), ignore_index=True)
#     return df



# def load(bucket, df, trg_key, trg_format):
#     target_key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
#     write_df_to_s3(bucket, df, target_key)
#     return True


# # def etl_report1(src_bucket, trg_bucket, date_list, trg_key, trg_format, columns, arg_date):
# #     df = extract(src_bucket, date_list)
# #     df = transform_report1(df, columns, arg_date)
# #     load(trg_bucket, df, trg_key, trg_format)

# df = extract(bucket, date_list)
# columns = ['ISIN', 'Mnemonic', 'Date', 'Time', 'StartPrice', 'EndPrice', 'MinPrice', 'MaxPrice', 'TradedVolume']
# arg_date = '2022-01-04'
# trg_key = 'xetra_daily_report_'
# trg_format = '.parquet'

# # #no bug

# def transform_report1(df, columns, arg_date):
#     df = df.loc[:, columns]
#     df.dropna(inplace=True)
#     df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
#     df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
#     df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
#     df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
#     df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
#     df.drop(columns=['prev_closing_price'], inplace=True)
#     df = df.round(decimals=2)
#     df = df[df.Date >= arg_date]
#     df.reset_index(inplace=True, drop=True)
#     return df



# trg_key = 'xetra_daily_report_'
# bucket_trg = s3.Bucket('xetra-112233')
# def etl_report1(src_bucket, trg_bucket, date_list, trg_key, trg_format, columns, arg_date):
#     df = extract(src_bucket, date_list)
#     df = transform_report1(df, columns, arg_date)
#     print(df)
#     load(trg_bucket, df, trg_key, trg_format)
#     return True

# etl_report1(bucket, bucket_trg, date_list, trg_key, trg_format, columns, arg_date)  

# # trg_bucket = s3.Bucket('xetra-112233')
# # etl_report1(bucket, trg_bucket, date_list, trg_key, trg_format, columns, arg_date)

In [ ]:
# run

main()

## Reading the uploaded file

In [ ]:
bucket_name_trg = 'xetra-112233'
s3 = boto3.resource('s3')
bucket_target = s3.Bucket(bucket_name_trg)

for obj in bucket_target.objects.all():
    print(obj.key)

In [ ]:
prq_obj_init = bucket_target.Object(key='xetra_daily_report_20210423_161441.parquet').get().get('Body').read()
data = BytesIO(prq_obj_init)
df_prq = pd.read_parquet(data)

In [ ]:
df_prq